In [1]:
%matplotlib widget
import cv2
import os
import json
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.patches as patches

## Manually rotate image ("horizon" correction)

In [2]:
meter_model = "wehrle"
camera_model = "espcam_120_deg"

meter_model_path = os.path.join("automatic_meter_reader", "meter_models", meter_model)
if not os.path.isdir(meter_model_path):
    os.mkdir(meter_model_path)

img_file = r"raw_images/cold_water_meter/cold_water_meter_20210208_224659.jpg"
rotation_angle_deg = -6

def rotate_image(image, angle):
    image_center = tuple((np.array(image.shape[1::-1]) - 1.0) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

with open(os.path.join("automatic_meter_reader", "cameras", "%s.json" % (camera_model)), "r") as f:
    camcalib = json.load(f)
camera_matrix = np.resize(np.array(camcalib["camera_matrix"]), (3, 3))
distortion_coefs = np.resize(np.array(camcalib["distortion_coefs"]), (1, -1))
new_camera_matrix = np.resize(np.array(camcalib["new_camera_matrix"]), (3, 3))

img = cv2.imread(img_file)
img = cv2.undistort(img, camera_matrix, distortion_coefs, None, new_camera_matrix)
img = rotate_image(img, rotation_angle_deg)

plt.figure(figsize=(6, 4))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axhline(215, lw=0.5, ls="--", c="r")
plt.tight_layout();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Define templates, register and dial ROIs

In [3]:
# Templates ROIs

template_rois = [
    (657, 220, 55, 38), # m3
    (84, 310, 60, 70), # Logo
    (340, 380, 160, 170), # Wheel
]

# Digits
register_roi = (178, 150, 420, 65)
num_digits = 8
digit_width = 40
digits_num_commas = 3
digits_num_integers = 4

## Generate config

In [4]:
config = {
    "name": meter_model,
    "templates": [],
    "register": {"digits": [] },
}

for i, (x0, y0, dx, dy) in enumerate(template_rois):
    template_img = img[y0:y0+dy, x0:x0+dx]
    template_img_file = "template_%d.jpg" % (i)
    cv2.imwrite(os.path.join(meter_model_path, template_img_file), template_img)
    config["templates"].append({
        "roi": [x0, y0, dx, dy],
        "file": template_img_file,
    })

x0, y0, dx, dy = register_roi
config["register"]["roi"] = list(register_roi)
for i in range(num_digits):
    spacing_x = (dx - num_digits * digit_width) / (num_digits - 1)
    nx0 = int(x0 + i * (digit_width + spacing_x))
    config["register"]["digits"].append({
        "roi": [nx0, y0, digit_width, dy],
        "multiplier": 10.0 ** (digits_num_integers - i)
    })

## Visualize

In [5]:
plt.figure(figsize=(6, 4))
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

# Templates
for template_conf in config["templates"]:
    x0, y0, dx, dy = template_conf["roi"]
    template_img = img[y0:y0+dy, x0:x0+dx]
    plt.gca().add_patch(patches.Rectangle((x0 - 0.5, y0 - 0.5), dx, dy, linewidth=1.0, ls="-", edgecolor="b", alpha=1.0, facecolor="none"))
    
# Register
x0, y0, dx, dy = config["register"]["roi"]
plt.gca().add_patch(patches.Rectangle((x0 - 0.5, y0 - 0.5), dx, dy, linewidth=1.0, ls="-", edgecolor='r', alpha=0.5, facecolor='none'))
for digit_conf in config["register"]["digits"]:
    x0, y0, dx, dy = digit_conf["roi"]
    plt.gca().add_patch(patches.Rectangle((x0 - 0.5, y0 - 0.5), dx, dy, linewidth=0.5, ls="--", edgecolor='r', alpha=1.0, facecolor='none'))
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Save config

In [64]:
with open(os.path.join(meter_model_path, "meter_config.json"), "w") as f:
    json.dump(config, f, indent=4)